In [2]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append('..') 
import platform
import os
from pathlib import Path
import shutil
from pathlib import Path
import shutil
from utils.behavior.session_utils import load_session_df, parse_session_string
from utils.behavior.lick_analysis import clean_up_licks, parse_lick_trains
from scipy.io import loadmat
from itertools import chain
from matplotlib import pyplot as plt
from IPython.display import display
from scipy.signal import find_peaks
from harp.clock import align_timestamps_to_anchor_points
from utils.basics.data_org import *
from utils.photometry.preprocessing import * 
from utils.photometry.plot_utils import align_signal_to_events, color_gradient, plot_FP_with_licks, plot_G_vs_Iso, plot_FP_beh_analysis
import numpy as np
from scipy.signal import butter, filtfilt, medfilt, sosfiltfilt
from scipy.optimize import curve_fit
import json
from sklearn.linear_model import LinearRegression
from matplotlib.gridspec import GridSpec
import pickle
from aind_fip_dff.utils.preprocess import batch_processing, tc_triexpfit
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import time
from contextlib import redirect_stdout
# %matplotlib inline
# %matplotlib widget

In [ ]:
# FP data
session = 'm743986d20241227'
session_dir = parse_session_string(session)
session_df, licks_L, licks_R = load_session_df(session)
signal, params = get_FP_data(session)

In [ ]:
# ephys data
session = 'mZS061d20210405'
session_dir = parse_session_string(session)
session_df, licks_L, licks_R = load_session_df(session)
ephys_file = os.path.join(session_dir['sortedFolder'], f"{session}_opto_units.pkl")
with open(ephys_file, 'rb') as f:
    ephys_data = pickle.load(f)